In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [5]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.AGD(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="armijo")

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.43689537048339844
epoch:  1, loss: 0.05907290056347847
epoch:  2, loss: 0.04560737684369087
epoch:  3, loss: 0.041200727224349976
epoch:  4, loss: 0.03535321727395058
epoch:  5, loss: 0.033263131976127625
epoch:  6, loss: 0.03005622699856758
epoch:  7, loss: 0.029424870386719704
epoch:  8, loss: 0.028712015599012375
epoch:  9, loss: 0.028429320082068443
epoch:  10, loss: 0.02814001403748989
epoch:  11, loss: 0.028067830950021744
epoch:  12, loss: 0.027942219749093056
epoch:  13, loss: 0.027717486023902893
epoch:  14, loss: 0.027663206681609154
epoch:  15, loss: 0.027647634968161583
epoch:  16, loss: 0.027562296018004417
epoch:  17, loss: 0.02752431109547615
epoch:  18, loss: 0.027462512254714966
epoch:  19, loss: 0.02742578461766243
epoch:  20, loss: 0.027356518432497978
epoch:  21, loss: 0.027337083593010902
epoch:  22, loss: 0.027305660769343376
epoch:  23, loss: 0.027239618822932243
epoch:  24, loss: 0.027230247855186462
epoch:  25, loss: 0.026729874312877655
epoc

In [6]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.AGD(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.05859089270234108
epoch:  1, loss: 0.057018253952264786
epoch:  2, loss: 0.04958638176321983
epoch:  3, loss: 0.0476008877158165
epoch:  4, loss: 0.03126613423228264
epoch:  5, loss: 0.030467860400676727
epoch:  6, loss: 0.027141306549310684
epoch:  7, loss: 0.026371723040938377
epoch:  8, loss: 0.025270327925682068
epoch:  9, loss: 0.02481643669307232
epoch:  10, loss: 0.0244156401604414
epoch:  11, loss: 0.02395530976355076
epoch:  12, loss: 0.023531224578619003
epoch:  13, loss: 0.0234550591558218
epoch:  14, loss: 0.023382103070616722
epoch:  15, loss: 0.02317837066948414
epoch:  16, loss: 0.022751931101083755
epoch:  17, loss: 0.022498449310660362
epoch:  18, loss: 0.022329850122332573
epoch:  19, loss: 0.02223997563123703
epoch:  20, loss: 0.021974263712763786
epoch:  21, loss: 0.021951016038656235
epoch:  22, loss: 0.0217050239443779
epoch:  23, loss: 0.021404843777418137
epoch:  24, loss: 0.021258600056171417
epoch:  25, loss: 0.020987771451473236
epoch:  26,